In [3]:
import dis

**Итераторы и итерируемые объекты**

Итератор, вообще говоря, это один из паттернов программирования. Подробнее о них можно почитать, например, в классическом учебнике "банды четырех" - Design Patterns. Мы не будем подробно на этом останавливаться

![iterator](iterator_pattern.gif)

Что нам действительно важно понять, это то, что **итератор** - это объект, который позволяет нам проходить по какому-то контейнеру - Aggregate - что-то, содержащее другие объекты. Чтобы быть итератором, классический итератор должен иметь операцию next, которая выдает следующий объект контейнера, и уметь сообщать об окончании итерации - в случае Python для этого служит специальный тип исключения - **StopIteration**

Объекты, которые позволяют получить итератор для них, называются итерируемыми. Чтобы получить итератор для них, в Python есть специальная функция **iter**

Например, всем известный список является итерируемым объектом. 

In [22]:
nucleotides = ["A", "T", "G", "C"]

nucleotides_iterator = iter(nucleotides)
print (type(nucleotides_iterator))

<class 'list_iterator'>


In [23]:
print (next(nucleotides_iterator))
print (next(nucleotides_iterator))
print (next(nucleotides_iterator))
print (next(nucleotides_iterator))

A
T
G
C


Последний вызов возвращает исключение

In [24]:
print (next(nucleotides_iterator))

StopIteration: 

Строки, множества, словари, файлы - тоже являются итерируемыми объектами. Зачем нам это нужно? 

На самом деле, даже использую цикл for мы неявно используем механизм итераторов

In [25]:
def print_nucleotides():
    for n in nucleotides:
        print (n)
        
print_nucleotides()

A
T
G
C


Наиболее явно нам это помогает увидеть машинный код, в который переводит Python нашу функцию

In [26]:
dis.dis(print_nucleotides)

  2           0 SETUP_LOOP              20 (to 22)
              2 LOAD_GLOBAL              0 (nucleotides)
              4 GET_ITER
        >>    6 FOR_ITER                12 (to 20)
              8 STORE_FAST               0 (n)

  3          10 LOAD_GLOBAL              1 (print)
             12 LOAD_FAST                0 (n)
             14 CALL_FUNCTION            1
             16 POP_TOP
             18 JUMP_ABSOLUTE            6
        >>   20 POP_BLOCK
        >>   22 LOAD_CONST               0 (None)
             24 RETURN_VALUE


Если это сложно, то for приблизительно можно записать вот так (только приблизительно, по скорости и красоте это проигрывает)

In [ ]:
def print_nucleotides_trivial():
    nucleotides_iterator = iter(nucleotides)
    while True:
        try:
            value = next(nucleotides_iterator)
        except StopIteration:
            break
        print (value)

print_nucleotides_trivial()

In [20]:
COMPLEMENT_TABLE = {"A" : "T", "C" : "G", "T" : "A", "G" : "C"}

Что будет выдавать нам итератор по словарю?

In [64]:
%%time
dt_iterator = iter(COMPLEMENT_TABLE)
print(dt_iterator)
while True:
    try:
        value = next(dt_iterator)
    except StopIteration:
        break
    print (value)


A
C
T
G
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 618 µs


In [42]:
%%time
for i in COMPLEMENT_TABLE.keys():
    print(i)

A
C
T
G
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 308 µs


List comprehensions. Списочные сокращения  

Часто работа с данными представимы в последовательной обработкой списков, в ходе которой к каждому объекту списка применяется какая функция, а полученный результат складывается в новый список. 

In [139]:
CODON_TABLE = {'AAA': 'K',
 'AAC': 'N', 'AAG': 'K', 'AAU': 'N', 'ACA': 'T',
 'ACC': 'T', 'ACG': 'T', 'ACU': 'T','AGA': 'R',
 'AGC': 'S', 'AGG': 'R', 'AGU': 'S', 'AUA': 'I',
 'AUC': 'I', 'AUG': 'M', 'AUU': 'I', 'CAA': 'Q',
 'CAC': 'H', 'CAG': 'Q', 'CAU': 'H', 'CCA': 'P',
 'CCC': 'P', 'CCG': 'P', 'CCU': 'P', 'CGA': 'R',
 'CGC': 'R', 'CGG': 'R', 'CGU': 'R', 'CUA': 'L',
 'CUC': 'L', 'CUG': 'L', 'CUU': 'L', 'GAA': 'E',
 'GAC': 'D', 'GAG': 'E', 'GAU': 'D', 'GCA': 'A',
 'GCC': 'A', 'GCG': 'A', 'GCU': 'A', 'GGA': 'G',
 'GGC': 'G', 'GGG': 'G', 'GGU': 'G', 'GUA': 'V',
 'GUC': 'V', 'GUG': 'V', 'GUU': 'V', 'UAA': '*',
 'UAC': 'Y', 'UAG': '*', 'UAU': 'Y', 'UCA': 'S',
 'UCC': 'S', 'UCG': 'S', 'UCU': 'S', 'UGA': '*',
 'UGC': 'C', 'UGG': 'W', 'UGU': 'C', 'UUA': 'L',
 'UUC': 'F', 'UUG': 'L', 'UUU': 'F'}

In [ ]:
import random
random.seed(42)
codons = []
codon_names = list(CODON_TABLE.keys())
for _ in range(100):
    codons.append(random.choice(codon_names))

codon_names[0:10]
print(codons)

In [ ]:
def get_aminoacids(codons):
    aminoacids = []
    for c in codons:
        aminoacids.append(CODON_TABLE[c])
    return aminoacids
aminoacids = get_aminoacids(codons)
print(aminoacids)

In [ ]:
%%timeit
aminoacids = get_aminoacids(codons)

В Python существует специальный синтаксис для таких действий - list comprehensions. Но это не просто "синтаксический сахар". Это работает быстрее за счет внутренних оптимизаций. 

In [ ]:
def get_aminoacids_comprehension(codons):
    return [CODON_TABLE[c] for c in codons]

In [ ]:
%%timeit
get_aminoacids_comprehension(codons)

Посмотрим на байткод (внутренне представления Python) наших двух функций

In [ ]:
from dis import dis
print ("Start of trivial version")
dis(get_aminoacids)
print ("End")
print ("Start of list comprehension version")
dis(get_aminoacids_comprehension)
print ("End")

Две наиболее частые задачи, которые применяются к спискам - это применить  функцию к значениям из одного списка и поместить результат в другой, и - отфильтровать значения из списка по какому-то критерию. Они могут быть реализованы с помощью двух подходов.
Первый - уже разобранный list comprehensions. Второй - использование встроенных функций map и filter

Описание второй функции в три раза меньше. Обычно, для чего Python сгенерил меньшее описаниие, быстрее и выполняется

Например, если хотим сделать словарь квадратов от 0 до 100. Можно так:

In [ ]:
squares = [x ** 2 for x in range(101)]
print (squares)

А можно так:

In [ ]:
squares = list(map(lambda x : x ** 2, range(101)))
print (squares)

Map принимает на вход первым аргументом функцию, которую надо применить к списку (или другому итерируемому объекту), а вторым- сам объект

In [ ]:
aminoacids = list(map(lambda x : CODON_TABLE[x], codons))
aminoacids[0:10]

In [ ]:
%%timeit
aminoacids = list(map(lambda x : CODON_TABLE[x], codons))

Работает медленнее, так как map возвращает итератор, который мы затем превращаем в список

Теперь допустим, что мы хотим отфильтровать наши объекты по какому-то принципу. Например, хотим из первых 1000 чисел отобрать только дающие при делении на 7 остаток 5

In [ ]:
div7_5 = [x for x in range(1000) if x % 7 == 5]
div7_5[0:10]

Аналогичную операцию делает filter

In [ ]:
div7_5 = list(filter(lambda x : x % 7 == 5, range(1000)))
print (div7_5[0:10])

Кроме того, есть reduce, который, из-за большого риска возникновения нечитаемого кода, был вынесен в functools и к использованию не рекомендуется

In [ ]:
from functools import reduce

In [ ]:
fact5 = reduce(lambda x,y : x * y, range(1, 10))
print (fact5)


product = lambda *args :reduce(lambda x, y : x * y, *args)
print (product(range(1, 6)))

Как он работает? 
Упрощенно вот так:

In [ ]:
def handmade_reduce(func, iterable, initial=None): # func should accept two positional arguments
    if initial is None:
        iterator = iter(iterable)
        try:
            acc = next(iterator) # initial value is the first element of iterable
        except Exception as e:
            print ("Reduce can't work with empty sequences")
            raise e
    else:
        acc = initial
        
    for value in iterator:
        acc = func(acc, value)
        
    return acc

In [ ]:
handmade_reduce(lambda x,y : x * y, range(1, 6))

Какие встроенные функции в Python ведут себя как reduce? 

**Тернарный оператор**

Важно, что в Python если однострочный if else

In [ ]:
a = 5 if 14 % 7 == 0 else 7
print (a)
codon = "UAA"
a = "StopCodon" if codon in ("UAA",  "UAG",  'UGA') else "Tryptophan" if codon == 'UGG' else "Other"
print (a)

И его тоже можно использовать в list comprehension, но смысл будет отличный от if после for x in range. Здесь каждому эдементу будет ставиться в соответствие значение, в соответствии с однострочным if

In [ ]:
odd_or_even = ["Odd" if x % 2 == 1 else "Even" for x in range(100)] 
odd_or_even[0:10]

Понятно, что в общем виде list comprehension можно записать так

**[func(arg) for arg in iterable <if cond(arg)>]**,


где func - какая-то функция от arg, в том числе это может быть тернарным оператором, а cond - некая функция, которая принимает arg и возвращает True или False (<> указывает на необязательность). Iterable - любой итерируемый объект

**Почему map, filter и reduce хороши в меру? (Как и вообще функциональное программирование в Python)**

Как проверить, является ли число 101 простым?


In [ ]:
if reduce(lambda x, y : 0 if x % y == 0 else x, range(2,101), 101):
    print ("Prime")
else:
    print ("Not prime")

Как обобщить на любое натуральное число

In [ ]:
is_prime = lambda z : False if z == 1 else True if reduce(lambda x,
                                                          y : 0 if x % y == 0 else x, range(2,z), z) else False
print (is_prime(10), is_prime(1), is_prime(2), is_prime(17))

Возьмем только простые числа из первых 1000

In [ ]:
primes = list(filter(lambda z : False if z == 1 else True if reduce(lambda x,
                                                          y : 0 if x % y == 0 else x, range(2,z), z) else False,
            range(1, 1001)))
print (primes[0:10])

Теперь оставим получим от них квадраты

In [ ]:
prime_squares = map(lambda x : x ** 2, (filter(lambda z : False if z == 1 else True if reduce(lambda x,
                                                          y : 0 if x % y == 0 else x, range(2,z), z) else False,
            range(1, 1001))))
print (list(prime_squares)[0:10])

И оставим их них те, что на втором месте имеют 2

In [ ]:
prime_squares_filtered = filter(lambda x : (x % 100) // 10  == 2, map(lambda x : x ** 2, (filter(lambda z : False if z == 1 else True if reduce(lambda x,
                            y : 0 if x % y == 0 else x, range(2,z), z) else False, range(1, 1001)))) )

print (list(prime_squares_filtered)[0:10])

А что, если мы хотим таким же образом создать словарь? Есть синтаксис и для словарей

Создадим словарь, для числа n хранящий значение соответствующего числа Фибоначчи



In [9]:
def fibonacci(n):
    f0, f1 = 0, 1
    for i in range(n):
        f0, f1 = f1, f0 + f1
    return f0

fibonacci_dt = { i : fibonacci(i) for i in range(10)}
print (fibonacci_dt)

{0: 0, 1: 1, 2: 1, 3: 2, 4: 3, 5: 5, 6: 8, 7: 13, 8: 21, 9: 34}


Можно и множество так сделать (пусть тоже чисел Фибоначчи), и tuple (но по приичинам, что обсудим позже, здесь придется использовать tuple до скобок)


In [10]:
fibonacci_set = {fibonacci(i) for i in range(10)}
fibonacci_tuple = tuple(fibonacci(i) for i in range(10))
print (fibonacci_set)
print (fibonacci_tuple)

{0, 1, 2, 3, 34, 5, 8, 13, 21}
(0, 1, 1, 2, 3, 5, 8, 13, 21, 34)


 Допустим, мы хотим перевести каждую строку списка в уникальный, только ей соответствующий номер. Очевидно, для этого нужно использовать словарь

In [17]:
import random
seq_arr = ["".join([random.choice("ATGC") for i in range(4)]) for i in range(100) ] # we use nested generators here
print(seq_arr)

['CTCC', 'TTCG', 'GAAT', 'TTGG', 'CCGA', 'CGCC', 'TTGA', 'CGCC', 'GCAA', 'GTCG', 'TTAA', 'CGTT', 'GAGC', 'GTTG', 'TCTC', 'GCTA', 'AGAA', 'ACAG', 'GAAT', 'CACC', 'GAGA', 'GCCC', 'GGAG', 'CGTT', 'GAAT', 'TACC', 'CCTC', 'ACGA', 'AGCA', 'TTGT', 'TCGT', 'CGAT', 'CTCA', 'CTCT', 'ATAT', 'GAAA', 'TAGA', 'ATTG', 'GGCA', 'GCAC', 'CCGT', 'CAGG', 'GATC', 'TGTT', 'AGCG', 'CATA', 'AGCT', 'TACA', 'GCCG', 'ATTA', 'GGCA', 'TTCT', 'CGAA', 'TGGT', 'CAAG', 'AAGC', 'TAGA', 'TCGG', 'ACCG', 'GCCA', 'TATG', 'GCAT', 'TACG', 'TCCG', 'TATC', 'AGCC', 'GAGT', 'ACCC', 'TCTT', 'GCGT', 'AGAG', 'GGTC', 'GAGG', 'CCCT', 'ATTC', 'TCCG', 'TCGA', 'GGTA', 'TATC', 'GACG', 'GGGC', 'AACG', 'ACCC', 'GTTG', 'TTAG', 'ACTC', 'AGGT', 'CCCC', 'AGCG', 'TCTG', 'TAAA', 'CTAT', 'TTTG', 'CGAT', 'CGAT', 'CCGT', 'GACT', 'GCAG', 'GCTG', 'GATC']


In [18]:
def variant1(seq_arr):
    mapping = {}
    seq_map = []
    for s in seq_arr:
        if s not in mapping:
            mapping[s] = len(mapping)
        seq_map.append(mapping[s])
    return seq_map
    
print (variant1(seq_arr)[0:10])

[0, 1, 2, 3, 4, 5, 6, 5, 7, 8]


Или, с использованием defaultdict

In [250]:
from collections import defaultdict

def variant2(seq_arr): 
    mapping = defaultdict(lambda : len(mapping))
    seq_map = []
    for s in seq_arr:
        if s not in mapping:
            mapping[s] = len(mapping)
        seq_map.append(mapping[s])
        
    return seq_map
    
print (variant2(seq_arr)[0:10])

[0, 1, 2, 3, 4, 5, 2, 6, 7, 8]


Задача - перевести список в список. Хотелось бы применить здесь map или list comprehension. Но как?


In [251]:
def variant3(seq_arr):
    mapping = defaultdict(lambda : len(mapping))
    seq_map = list(map(lambda x : mapping[x], seq_arr))
    return seq_map
    
print (variant3(seq_arr)[0:10])

[0, 1, 2, 3, 4, 5, 2, 6, 7, 8]


In [252]:
def variant4(seq_arr):
    mapping = defaultdict(lambda : len(mapping))
    seq_map = [mapping[s] for s in seq_arr]
    return seq_map
print (variant4(seq_arr)[0:10])

[0, 1, 2, 3, 4, 5, 2, 6, 7, 8]


In [253]:
%timeit variant1(seq_arr)
%timeit variant2(seq_arr)
%timeit variant3(seq_arr)
%timeit variant4(seq_arr)

13 ms ± 576 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
13.2 ms ± 232 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
12.1 ms ± 185 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5.58 ms ± 118 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**Еще немного о lambda-функциях и областях видимости.**

Lambda  функции во многом являются полноправными функциями, они могут принимать \*args, \*\*kwargs, как уже было показано. Могут они иметь и аргументы по-умолчанию. Зачем?

Что напечатает пример, приведенной ниже?

In [258]:
funcs_for_very_important_work = { ind : lambda x : x + ind for ind in range(10) }

value = 5
for func in funcs_for_very_important_work.values():
    print (func(value))

14
14
14
14
14
14
14
14
14
14


Дело в том, что при создании lambda-функции в данном примере происходит так называемое "ленивое вычисление" для ind, щапоминается ссылка на него, а не собственно значение. Из-за этого во всех функциях он становится равен 9 (последнее занчение, которое он принимает). Как это избежать? Нужно заставить функцию вычислить значение ind при создании. 

Наркоманский, хоть и работающий способ. Будем создавать нужную нам lambda функцию еще одной lambda-функцией, которую будем вызывать в момент создания. 

In [259]:
funcs_for_very_important_work = { ind : (lambda i : lambda x : x + i)(ind) for ind in range(10) }

value = 5
for func in funcs_for_very_important_work.values():
    print (func(value))

5
6
7
8
9
10
11
12
13
14


Хоть это и common practice в языках типа Golang, в Python можно сделать легче. Просто делаем ind аргументом по-умолчанию, он вычислится при создании функции

In [260]:
funcs_for_very_important_work = { ind : lambda x, i=ind : x + i for ind in range(10) }

value = 5
for func in funcs_for_very_important_work.values():
    print (func(value))

5
6
7
8
9
10
11
12
13
14


**Генераторы. Базовый уровень**

Что будет, если мы попытаемся сделать кортеж по аналогии со списком? (просто поменяв скобочки)

In [2]:
tuple_question = (x * 2 for x in range(10))
print (tuple_question, type(tuple_question))

<generator object <genexpr> at 0x7f66540b9fc0> <class 'generator'>


Какой-то неправильный кортеж

In [267]:
tuple_question[0]

TypeError: 'generator' object is not subscriptable

![generator](i-am-a-generator.jpg)

Объект, который мы получили называется генератор. В чем отличие его от списка/кортежа ? 

По нему точно так же можно итерироваться 

In [4]:
squares = (x * 2 for x in range(10))
for s in squares:
    print (s)

0
2
4
6
8
10
12
14
16
18


In [18]:
def my_gen(f0, f1):
    while True:
        yield f0
        f0, f1 = f1, f0 * 2
    pass
print(next(my_gen(1,1)))
print(next(my_gen(1, 1)))
print(next(my_gen(1,1)))

1
1
1


Один раз

In [90]:
for s in squares:
    print (s)

Генератор - один из способов реализации концепции ленивых вычислений. Мы пишем, что за вычисления нужно осуществить, а осуществлены они будут только тогда, когда это непосредственно потребуется.
Таким образом в памяти будет храниться лишь минимум информации, необходимой для осуществления этого вычисления.

In [104]:
%%time
squares = [x ** 2 for x in range(100000)]
for s in squares:
    if s % 99000000000 == 0:
        pass

CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 37.2 ms


In [97]:
%%time
squares_generator = (x ** 2 for x in range(100000))
for s in squares_generator:
    if s % 99000000000 == 0:
        pass

CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 42.3 ms


По скорости нет разности. Даже в случае с генератором чуть медленнее (это лишь пока мы не уперлись в лимит по памяти)

Для использования memory_profiler надо выполнить следующие команды

conda install -c chroxvi memory_profiler

conda install -c anaconda psutil

In [105]:
%load_ext memory_profiler

In [106]:
%%memit
squares = [x ** 2 for x in range(10000000)]
for s in squares:
    if s % 99000000000 == 0:
        print (s)
    


0
10890000000000
43560000000000
98010000000000
peak memory: 426.64 MiB, increment: 382.52 MiB


In [108]:
del squares

NameError: name 'squares' is not defined

In [109]:
%%memit
squares_generator = (x ** 2 for x in range(10000000))
for s in squares_generator:
    if s % 99000000000 == 0:
        print (s)

0
10890000000000
43560000000000
98010000000000
peak memory: 40.90 MiB, increment: 0.00 MiB


Видим, что использование памяти в случае с генератором минимально, в случае списка мы же получаем огромный оверхэд по памяти.

Когда нам уже случалось использовать генератор? На самом деле при чтении файла с помощью конструкции for line in file мы используем именно генератор - файл не подгружается полностью, а частями, такими, чтобы минимизировать использование памяти, но при этом не замедлить исполняемый код

Для использования кода ниже, если у вас Windows просто загрузите файл по [ссылке](https://www.genome.wisc.edu/pub/sequence/U00096.2.fas)

In [75]:
!wget https://www.genome.wisc.edu/pub/sequence/U00096.2.fas

--2018-03-18 17:10:22--  https://www.genome.wisc.edu/pub/sequence/U00096.2.fas
Распознаётся www.genome.wisc.edu (www.genome.wisc.edu)... 128.104.81.186
Подключение к www.genome.wisc.edu (www.genome.wisc.edu)|128.104.81.186|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа... 200 OK
Длина: 4697740 (4,5M) [text/plain]
Сохранение в каталог: ««U00096.2.fas.1»».

U00096.2.fas.1      100%[===================>]   4,48M  59,7KB/s    in 4m 46s  

2018-03-18 17:15:50 (16,1 KB/s) - «U00096.2.fas.1» сохранён [4697740/4697740]



In [110]:
path_to_fasta_file = "U00096.2.fas"

In [111]:
from collections import Counter

In [112]:
cnt = Counter()
?cnt.update

In [119]:
%%memit
def process_line(line, cnt):
    cnt.update(line)

global_cnt = Counter()
with open(path_to_fasta_file) as in_file:
    print(in_file.readline())
    for line in in_file:
        process_line(line, global_cnt)
print (global_cnt.most_common())

>U00096.2  Escherichia coli K-12 MG1655 complete genome (4639675 bp)

[('c', 1179554), ('g', 1176923), ('a', 1142228), ('t', 1140970), ('\n', 57996)]
peak memory: 40.78 MiB, increment: 0.00 MiB


In [120]:
del in_file
del global_cnt

Сравним просто с зачитыванием всего файла в память

In [121]:
%%memit
def process_line(line, cnt):
    cnt.update(line)

global_cnt = Counter()
with open(path_to_fasta_file) as in_file:
    in_file.readline()
    lines = in_file.readlines()
    for line in lines:
        process_line(line, global_cnt)
print (global_cnt.most_common())

[('c', 1179554), ('g', 1176923), ('a', 1142228), ('t', 1140970), ('\n', 57996)]
peak memory: 47.04 MiB, increment: 6.26 MiB


In [122]:
del in_file
del global_cnt

Генераторы можно использовать при создании других генераторов. При этом ни на одном из этапов мы не разворачиваем в памяти весь требуемый список, потому использование памяти минимально

In [132]:
%%memit
squares = (x ** 2 for x in range(1000))
squares_with_1_on_2 = (x for x in squares if (x % 100) // 10 == 1)
squares_with_1_on_2_div7 = (x for x in squares_with_1_on_2 if x % 7 == 0)
squares_with_1_on_2_div7_add3 = map(lambda x : x + 3, squares_with_1_on_2_div7)

sm = sum(squares_with_1_on_2_div7_add3)

peak memory: 47.12 MiB, increment: 0.00 MiB


Генераторы - одно из мощнейших средств Python. Они позволяют, единожды написав какие-то обработчики данных, 
дальше использовать их, соединять друг с другом и тд без дополнительных затрат по памяти.

Более близкий пример - посчитать число разных кодонов в нуклеотидной последовательности, кодирующей белок

In [140]:
start_codon = 'AUG'
stop_codons = ['UAA','UAG', 'UGA']
coding_codons = list(set(CODON_TABLE) - set(stop_codons))
seq = start_codon +  "".join(random.choice(coding_codons) for _ in range(1000)) + random.choice(stop_codons)

In [151]:
codons = (seq[i:i+3] for i in range(0, len(seq), 3))
print(codons)
aminoacids = (CODON_TABLE[c] for c in codons)
print(aminoacids)
cnt = Counter(aminoacids)
print(cnt)
print (cnt.most_common())
print(len(cnt))
print (len(cnt.most_common()))

<generator object <genexpr> at 0x7f805b761048>
<generator object <genexpr> at 0x7f805b7610f8>
Counter({'S': 101, 'R': 101, 'L': 89, 'P': 70, 'G': 69, 'V': 65, 'A': 61, 'T': 56, 'I': 45, 'F': 43, 'N': 41, 'E': 39, 'K': 36, 'C': 35, 'Q': 33, 'Y': 32, 'D': 29, 'H': 25, 'W': 16, 'M': 15, '*': 1})
[('S', 101), ('R', 101), ('L', 89), ('P', 70), ('G', 69), ('V', 65), ('A', 61), ('T', 56), ('I', 45), ('F', 43), ('N', 41), ('E', 39), ('K', 36), ('C', 35), ('Q', 33), ('Y', 32), ('D', 29), ('H', 25), ('W', 16), ('M', 15), ('*', 1)]
21
21


Для работы с итерируемыми объектами, частности, с генераторами, существует множество встроенных функций

**enumerate** - принимает итератор и возвращает итератор, который возвращает кортежи из (индекс, элемент_исходного итератора)

In [ ]:
?enumerate

In [84]:
aminoacids = CODON_TABLE.values()

print (next(enumerate(aminoacids)))

for ind, ac in enumerate(aminoacids):
    print (ind, ac)
    if ind == 4:
        break


(0, 'K')
0 K
1 N
2 K
3 N
4 T


Можно задать начало отчета (удобно, когда хочется иметь нумерацию с 1, например)

In [85]:
start = 1
for ind, ac in enumerate(aminoacids, start):
    print (ind, ac)
    if ind == 4:
        break

1 K
2 N
3 K
4 N


In [86]:
?zip

**zip** - принимает итераторы, возвращает кортежи, где i элемент кортежа происходит из i итератора. 

In [89]:
for t in zip(range(3), range(3, 6)):
    print (t)
    
for i, j in zip(range(3, 6), range(6, 9)):
    print (i, j)    

(0, 3)
(1, 4)
(2, 5)
3 6
4 7
5 8


С ним надо быть аккуратным. Если какой-то итератор содержит больше значений, чем другие, то эти значения просто не покажутся

In [90]:
for i, j in zip(range(3, 6), range(6, 20)):
    print (i, j)   

3 6
4 7
5 8


**reversed** - возвращает итератор, идущий в обратную сторону по контейнеру (как и следует из названия). Полезен, например, тем, что обычно использующийся для списков способ создает новый список, а этот - нет

In [152]:
lst = [random.choice("ATGC") for i in range(10000000)]

In [153]:
%%memit
for i in lst[::-1]:
    if i == "G":
        break

peak memory: 200.66 MiB, increment: 76.23 MiB


In [154]:
%%memit
for i in reversed(lst):
    if i == "G":
        break

peak memory: 124.43 MiB, increment: 0.00 MiB


Пример использования

In [157]:
def rindex(lst, query):
    for ind, val in enumerate(reversed(lst)):
        print(val, query)
        if val == query:
            return len(lst) - ind - 1
    return -1

In [158]:
rindex([1,2,3,10,15,4], 3)

4 3
15 3
10 3
3 3


2

**Itertools**

Много функций для работы с итераторами содержатся и в стандартном пакете

In [160]:
from itertools import count, cycle, repeat, chain, compress, dropwhile, takewhile, islice

**count** - бесконечный счетчик

In [130]:
cnt = count(5)
for _ in range(7):
    print (next(cnt))

5
6
7
8
9
10
11


**cycle** - бесконечный циклический итератор

In [131]:
frames = (0, 1, 2)
cycle_frames = cycle(frames)
for _ in range(7):
    print (next(cycle_frames))

0
1
2
0
1
2
0


**repeat** - просто итератор, который бесконечно возвращает одно и то же значение

Документация говорит, что полезно использовать, например, как источник константных значений для zip

In [163]:
list(map(pow, range(10), repeat(2)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

**chain** - принимает несколько итераторов и объединяет их в один

In [167]:
for i in chain(range(5,7), range(17, 19)):
    print (i)

5
6
17
18


**chain.from_iterable** - делает то же самое, но принимает один итератор, который возвращает другие итераторы

In [137]:
for i in chain.from_iterable(range(1, x) for x in range(3, 6)):
    print (i)

1
2
1
2
3
1
2
3
4


**compress** - принимает на вход итератор со значениями и итератор с булевыми переменными, обозначающими брать нужную переменную или нет

In [139]:
for i in compress("ATGC", [True, False, True, False]):
    print (i)

A
G


**dropwhile** - не берет значения из итератора, пока выполняется условие, после этого возвращает остаток

In [155]:
list(dropwhile(lambda y : y < 255000, (sum(i** 2 for i in range(0, x)) for x in range(5, 100)) ))

[255346, 263810, 272459, 281295, 290320, 299536, 308945, 318549]

**takewhile** - наоборот, берет значения пока выполняется условие

In [159]:
list(takewhile(lambda y : y < 150, (sum(i** 2 for i in range(0, x)) for x in range(5, 100)) ))

[30, 55, 91, 140]

**islice** - взять срез по итератору

In [291]:
a = (x ** 2 for x in range(10))
print (list(islice(a, 0, 3)))
print (list(islice(a, 0, 3)))

[0, 1, 4]
[9, 16, 25]


Отдельно стоят итераторы, позволяющие делать всякие комбинаторные вещи

In [161]:
from itertools import product, combinations, permutations, combinations_with_replacement

**product** - декартово произведение

In [165]:
list(product(range(3), range(3))) 

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

**combinations**

In [168]:
list(combinations("ATGC", 2))

[('A', 'T'), ('A', 'G'), ('A', 'C'), ('T', 'G'), ('T', 'C'), ('G', 'C')]

и т.д:)

**Многострочные генераторы**

До данного момента мы с вами писали только однострочные генераторы. Одна часто хочется написать какую-то сложную функцию, которая работает по тому же принципу, тратя мало памяти на вычисления. Для этого существует специальный синтаксис - **yield**

Напишем, например, генератор чисел Фибоначчи

In [1]:
def fibonacci_gen():
    f0, f1 = 0, 1
    while True:
        yield f0
        f0, f1 = f1, f0 + f1
        print(f0)

In [2]:
fib_gen = fibonacci_gen()
for i in range(5):
    print (next(fib_gen))

0
1
1
1
1
2
2
3
3


In [174]:
fib_gen = fibonacci_gen()
list(takewhile(lambda x : x < 100, fib_gen))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]

После выполнения команды **yield**, функция возвращает значение f0, но она завершается, а только замирает. Следующий вызов к ней "пробуждает функцию", она делает еще одну итерацию бесконечного цикла и снова замирает, возвращая значение


Можем написать функцию, которая вернет нам итераторы по всем белкам на данной цепи (будем считать, что белки не могут быть вложенными)

In [409]:
def transcribe(n, tr_table={"A" : "A", "T" : "U", "G" : "G", "C" : "C"}):
    return tr_table[n]

def iter_seqs(in_file_path):
    open_frames = cycle([0,1,2])
    proteins = [None, None, None]
    with open(path_to_fasta_file) as in_file:
        in_file.readline()
        letters = chain.from_iterable((line for line in in_file))
        dna_nucleotides = filter(lambda x : x != "\n", letters)
        nucleotides = map(lambda x : transcribe(x.upper()), dna_nucleotides)
        
        triplet = "".join(islice(nucleotides, 0,3))
        
        for n in nucleotides:
            open_frame = next(open_frames)
            cur_prot = proteins[open_frame]
            ac = CODON_TABLE[triplet]
            if ac == "*":
                if cur_prot:
                    proteins[open_frame] = None
                    cur_prot.append(triplet)
                    yield cur_prot
            elif ac == "M" and cur_prot is None:
                proteins[open_frame] = [triplet]
            elif cur_prot is not None:
                cur_prot.append(triplet)
            triplet = triplet[1:] + n
        

In [410]:
with open(path_to_fasta_file) as in_file:
    in_file.readline()
    print (in_file.readline())

agcttttcattctgactgcaacgggcaatatgtctctgtgtggattaaaaaaagagtgtctgatagcagcttctgaactg



In [411]:
%%memit -i 0.1 -c -o
prots_gen = iter_seqs(path_to_fasta_file)
codon_usage_cnt = Counter()
for prot_codons in prots_gen:
    codon_usage_cnt.update(prot_codons)

peak memory: 211.86 MiB, increment: 6.53 MiB


<MemitResult : peak memory: 211.86 MiB, increment: 6.53 MiB>

In [412]:
print (codon_usage_cnt.most_common())

[('AUG', 76237), ('CUG', 53366), ('GCG', 46843), ('AAA', 44178), ('CAG', 43663), ('CGC', 40163), ('AUC', 39387), ('GAA', 39022), ('GCC', 38774), ('UUU', 37874), ('GGC', 36781), ('AUU', 36398), ('GAU', 35561), ('ACC', 33525), ('CCG', 33307), ('UUC', 33123), ('AAC', 32709), ('GCA', 32215), ('GUG', 31416), ('AAU', 30160), ('AGC', 29603), ('GGU', 29316), ('UUG', 28908), ('GUU', 28896), ('ACG', 28215), ('UGG', 28080), ('UGC', 26329), ('CGU', 26240), ('GCU', 25462), ('CCA', 24276), ('UUA', 24116), ('CAU', 23838), ('CGG', 23805), ('GUC', 23643), ('AAG', 23143), ('UCG', 23086), ('CAA', 22849), ('UCA', 22774), ('GAC', 22113), ('CAC', 21484), ('AUA', 21081), ('GUA', 19901), ('GAG', 19893), ('UAU', 19577), ('UCC', 18734), ('CUU', 18611), ('UGA', 18311), ('UAC', 17833), ('ACA', 17638), ('CUC', 17231), ('AGU', 16749), ('GGG', 16321), ('ACU', 15999), ('UCU', 15969), ('GGA', 15646), ('UAA', 15573), ('CCC', 15446), ('CGA', 15273), ('UGU', 15088), ('CCU', 14393), ('AGA', 14140), ('AGG', 13613), ('CUA',

Что дают генераторы:

![generators](generators_pipelines.png)

**Coroutines**

Начиная с Python2.5 появилась возможность не только получать значение из генератора, но и посылать их туда. Это плавно дает возможность использовать генератор как корутину - сопрограмму. То есть можно сделать набор функций, общающихся друг с другом и выдающим в конце результат результат. 

Общение происходит с помощью функции генератора send. Чтобы иметь возможность посылать корутине значения, ее надо "запустить", послав в нее None или применив к ней next.

После окончания общения с корутиной ее надо закрыть, при этом в самой корутине можно прописать реакцию на закрытие (с помощью exception, если его не перехватывать, то все равно ничего не произойдет) 

Например, напишем свой grep

In [5]:
def grep_simple(pattern): # also works, but doesn't  Goodbye:(
    print ("Looking for %s" % pattern)
    while True:
        line = (yield)
        if pattern in line:
            print (line)


def grep(pattern):
    print ("Looking for %s" % pattern)
    try:
        while True:
            line = (yield)
            if pattern in line:
                print (line)
    except GeneratorExit:
        print ("Going away. Goodbye")

# Example use
#g = grep_simple("python")
g = grep("pyth")
g.send(None)
g.send("Yeah, but no, but yeah, but no")
g.send("A series of tubes")
g.send("python generators rock!")
g.close() 

Looking for pythd
Going away. Goodbye


Таким образом, меняется наша парадигма - раньше мы сцепляли генераторы друг с другом и последний из них "вытаскивал" данные. Теперь мы можем посылать данные в корутину из другой корутины или основной программы

![coroutines](coroutines.png)

Чтобы не заставлять себя каждый раз посылать "холодному" генератору None, можно написать декоратор

In [323]:
def coroutine(func):
    def start(*args,**kwargs):
        cr = func(*args,**kwargs)
        cr.send(None)
        return cr
    return start

В чем бонусы coroutine? С помощью них можно реализовывать конечные автоматы

![automata](automata.png)

А еще мы можем передавать наши значения сразу в несколько мест! Генераторы в таких случаях бы исчерпались

![coroutines_branching](coroutines_branching.png)

Напишем например, более совершенную версию программы, выдающей все белки

In [398]:
def transcribe(n, tr_table={"A" : "A", "T" : "U", "G" : "G", "C" : "C"}):
    return tr_table[n]

@coroutine
def get_triplets_reader(receiver, ignore_first=0):
    nucleotides = []
    for i in range(ignore_first):
        n = (yield)
        #print ("Triplet reader", ignore_first, n)
        
    while True:
        n = (yield)
        #print ("Triplet reader", ignore_first, n)
        nucleotides.append(n)
        if len(nucleotides) == 3:
            #print ("sending")
            triplet = "".join(nucleotides)
            nucleotides = []
            receiver.send(triplet)
            
@coroutine
def get_proteins_reader(protein_receiver, codons_receiver, name):
    triplets = None
    aminoacids = None
    while True:
        t = (yield)
        
        ac = CODON_TABLE[t]
        #print ("Protein reader", name, t, ac)
        if ac == "M" and triplets is None:
            triplets = [t]
            aminoacids = [ac]
        elif ac == "*" and triplets is not None:
            triplets.append(t)
            protein_receiver.send("".join(aminoacids))
            codons_receiver.send(triplets)
            triplets = None
            aminoacids = None
        elif triplets is not None:
            triplets.append(t)
            aminoacids.append(ac)
            
@coroutine
def get_triplets_counter(cnt):
    while True:
        codons = (yield)
        cnt.update(codons)

@coroutine
def get_proteins_writer(lst):
    while True:
        protein = (yield)
        lst.append(protein)
            

def iter_seqs(in_file_path):
    with open(in_file_path) as in_file:
        in_file.readline()
        letters = chain.from_iterable((line for line in in_file))
        dna_nucleotides = filter(lambda x : x != "\n", letters)
        nucleotides = map(lambda x : transcribe(x.upper()), dna_nucleotides)
       
        codons_cnt = Counter()
        triplets_counter = get_triplets_counter(codons_cnt)
        proteins_lst = []
        proteins_writer = get_proteins_writer(proteins_lst)
        orfs_readers = [get_triplets_reader(get_proteins_reader(proteins_writer,
                                                           triplets_counter, start_pos), 
                                       ignore_first=start_pos)\
                        for start_pos in range(3)]

        for n in nucleotides:
            for reader in orfs_readers:
                reader.send(n)
        
    return proteins_lst, codons_cnt 
        

In [399]:
path_to_simple_fasta_file = "example.fasta"
with open(path_to_simple_fasta_file, "w") as out_file:
    out_file.write(">example\n")
    out_file.write("ATGTAAATGTAA")

In [404]:
proteins_lst, codons_cnt = iter_seqs(path_to_fasta_file)

In [408]:
codons_cnt

Counter({'AAA': 44178,
         'AAC': 32709,
         'AAG': 23143,
         'AAU': 30160,
         'ACA': 17638,
         'ACC': 33525,
         'ACG': 28215,
         'ACU': 15999,
         'AGA': 14140,
         'AGC': 29603,
         'AGG': 13613,
         'AGU': 16749,
         'AUA': 21081,
         'AUC': 39387,
         'AUG': 76237,
         'AUU': 36398,
         'CAA': 22849,
         'CAC': 21484,
         'CAG': 43663,
         'CAU': 23838,
         'CCA': 24276,
         'CCC': 15446,
         'CCG': 33307,
         'CCU': 14393,
         'CGA': 15273,
         'CGC': 40163,
         'CGG': 23805,
         'CGU': 26240,
         'CUA': 7489,
         'CUC': 17231,
         'CUG': 53366,
         'CUU': 18611,
         'GAA': 39022,
         'GAC': 22113,
         'GAG': 19893,
         'GAU': 35561,
         'GCA': 32215,
         'GCC': 38774,
         'GCG': 46843,
         'GCU': 25462,
         'GGA': 15646,
         'GGC': 36781,
         'GGG': 16321,
         'GG

In [407]:
proteins_lst[0:10]

['MSLCGLKKECLIAASELVTCRE',
 'MKRISTTITTTITITTGNGAG',
 'MQNVFCVLPIFWKAMPGRGRWPPSSLPPPKSPTTWWR',
 'MLYPISAMPNVFLPNF',
 'MSCMALVCWGSARIASTLR',
 'MKKANWWCLDATVPTTLLRCWLPVYAPIVARFGRTLTGSIPATRVRCPMRGC',
 'MKTNYRSRAFPI',
 'MVCAPCVGSRRNSLPHWPAPISTLSPLLRDLLNAQSLSW',
 'MMRPLACALLIRCCSIPIRLSKCL',
 'MYMALIWKTGRKNWRKPKSRLISGA']

In [406]:
codons_cnt

Counter({'AAA': 44178,
         'AAC': 32709,
         'AAG': 23143,
         'AAU': 30160,
         'ACA': 17638,
         'ACC': 33525,
         'ACG': 28215,
         'ACU': 15999,
         'AGA': 14140,
         'AGC': 29603,
         'AGG': 13613,
         'AGU': 16749,
         'AUA': 21081,
         'AUC': 39387,
         'AUG': 76237,
         'AUU': 36398,
         'CAA': 22849,
         'CAC': 21484,
         'CAG': 43663,
         'CAU': 23838,
         'CCA': 24276,
         'CCC': 15446,
         'CCG': 33307,
         'CCU': 14393,
         'CGA': 15273,
         'CGC': 40163,
         'CGG': 23805,
         'CGU': 26240,
         'CUA': 7489,
         'CUC': 17231,
         'CUG': 53366,
         'CUU': 18611,
         'GAA': 39022,
         'GAC': 22113,
         'GAG': 19893,
         'GAU': 35561,
         'GCA': 32215,
         'GCC': 38774,
         'GCG': 46843,
         'GCU': 25462,
         'GGA': 15646,
         'GGC': 36781,
         'GGG': 16321,
         'GG

In [403]:
CODON_TABLE

{'AAA': 'K',
 'AAC': 'N',
 'AAG': 'K',
 'AAU': 'N',
 'ACA': 'T',
 'ACC': 'T',
 'ACG': 'T',
 'ACU': 'T',
 'AGA': 'R',
 'AGC': 'S',
 'AGG': 'R',
 'AGU': 'S',
 'AUA': 'I',
 'AUC': 'I',
 'AUG': 'M',
 'AUU': 'I',
 'CAA': 'Q',
 'CAC': 'H',
 'CAG': 'Q',
 'CAU': 'H',
 'CCA': 'P',
 'CCC': 'P',
 'CCG': 'P',
 'CCU': 'P',
 'CGA': 'R',
 'CGC': 'R',
 'CGG': 'R',
 'CGU': 'R',
 'CUA': 'L',
 'CUC': 'L',
 'CUG': 'L',
 'CUU': 'L',
 'GAA': 'E',
 'GAC': 'D',
 'GAG': 'E',
 'GAU': 'D',
 'GCA': 'A',
 'GCC': 'A',
 'GCG': 'A',
 'GCU': 'A',
 'GGA': 'G',
 'GGC': 'G',
 'GGG': 'G',
 'GGU': 'G',
 'GUA': 'V',
 'GUC': 'V',
 'GUG': 'V',
 'GUU': 'V',
 'UAA': '*',
 'UAC': 'Y',
 'UAG': '*',
 'UAU': 'Y',
 'UCA': 'S',
 'UCC': 'S',
 'UCG': 'S',
 'UCU': 'S',
 'UGA': '*',
 'UGC': 'C',
 'UGG': 'W',
 'UGU': 'C',
 'UUA': 'L',
 'UUC': 'F',
 'UUG': 'L',
 'UUU': 'F'}

Если вам интересно узнать больше о корутинах и Python - отличный сайт http://www.dabeaz.com/tutorials.html.
Про корутины я брал с курса "A Curious Course on Coroutines and Concurrency"